In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from transformers import AutoProcessor, AutoModelForImageClassification
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
file_path = "/users/PAS2136/rayees/CV4A - Efficient Probing/ProbingSubsets/probing_5800_ratio.csv"
df = pd.read_csv(file_path)
cols = ['ImageFilePath', 'ScientificName']
df = df[cols]
df = df.dropna()
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Platyderus depressus
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Nebria brevicollis


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "microsoft/swinv2-base-patch4-window8-256"

processor = AutoProcessor.from_pretrained(model_name, use_fast=True)

model = AutoModelForImageClassification.from_pretrained(model_name).to(device).eval()


In [5]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        features = outputs.logits
    
    return features.cpu().numpy()


In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing Images ...") ])

seed = 99

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X, y, df_indices, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)


Processing Images ...:   0%|          | 0/5800 [00:00<?, ?it/s]

In [7]:

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

metrics = {}

predictions = {}

for name, model in tqdm(models.items(), desc="Training Models", leave=True):

    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")

metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 12.93% | Prec: 13.90% | Rec: 12.93% | F1: 11.23% | Bal Acc: 11.64% | MCC: 0.1256
LogisticRegression        | Acc: 49.31% | Prec: 49.04% | Rec: 49.31% | F1: 47.40% | Bal Acc: 41.96% | MCC: 0.4904
NearestNeighbor           | Acc: 19.66% | Prec: 20.84% | Rec: 19.66% | F1: 18.36% | Bal Acc: 14.90% | MCC: 0.1924
MLP_Baseline              | Acc: 45.09% | Prec: 45.16% | Rec: 45.09% | F1: 43.59% | Bal Acc: 38.57% | MCC: 0.4480


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})

In [10]:
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
1548,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Loricera pilicornis,Cicindela campestris,Dyschirius salinus,Abax parallelepipedus,Amara aenea
4081,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Amara familiaris,Ophonus rufibarbis,Amara bifrons,Agonum muelleri,Amara familiaris


In [11]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.12931,0.139026,0.12931,0.112251,0.116414,0.125589
LogisticRegression,LogisticRegression,0.493103,0.490389,0.493103,0.474022,0.419648,0.490384
NearestNeighbor,NearestNeighbor,0.196552,0.208395,0.196552,0.183602,0.149016,0.192356
MLP_Baseline,MLP_Baseline,0.450862,0.451599,0.450862,0.435872,0.385674,0.447979


In [12]:

basedir = '/users/PAS2136/rayees/CV4A - Efficient Probing/BritishCarabids_5800_ratio/'


In [13]:

test_df.to_csv(f"{basedir}/5.SWINv2-species.csv", index=False)

metrics_df.to_csv(f"{basedir}/5.SWINv2-species-metrics.csv", index=False)

